In [1]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline


plt.style.use('ggplot')

c:\Users\Morgan\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
subway_df_weekend = pd.read_csv('../data/2014-2019_weekend Subway Tables_UL.csv')
subway_df_weekend.head(5)

,Station (alphabetical by borough),Unnamed: 1,Boro,2014,2015,2016,2017,2018,2019,2018-2019 Change,Unnamed: 10,2019 Rank
0,The Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"138 St-Grand Concourse (4,5)",1.0,Bx,"3,017","3,136","3,123","2,793","2,557","3,537",980,38.30%,337.0
2,"149 St-Grand Concourse (2,4,5)",NaN,Bx,"15,326","13,683","12,797","12,623","11,873","11,647",-226,-1.90%,148.0
3,"161 St-Yankee Stadium (B,D,4)",NaN,Bx,"36,176","36,969","35,171","33,398","32,173","31,627",-546,-1.70%,29.0
4,167 St (4),NaN,Bx,"10,417","11,942","11,579","9,981","9,822","8,986",-836,-8.50%,178.0


In [6]:
# Replace Boro code with borough
subway_df_weekend['Boro'].replace('Bx', 'The Bronx', inplace=True)
subway_df_weekend['Boro'].replace('B', 'Brooklyn', inplace=True)
subway_df_weekend['Boro'].replace('M', 'Manhattan', inplace=True)
subway_df_weekend['Boro'].replace('Q', 'Queens', inplace=True)
subway_df_weekend['Boro'].value_counts()


Brooklyn     157
Manhattan    121
Queens        78
The Bronx     68
Name: Boro, dtype: int64

In [7]:
# Delete rows with no value
subway_df_weekend.dropna(axis=0, thresh=3, inplace=True)
subway_df_weekend.head(5)

,Station (alphabetical by borough),Unnamed: 1,Boro,2014,2015,2016,2017,2018,2019,2018-2019 Change,Unnamed: 10,2019 Rank
1,"138 St-Grand Concourse (4,5)",1.0,The Bronx,"3,017","3,136","3,123","2,793","2,557","3,537",980,38.30%,337.0
2,"149 St-Grand Concourse (2,4,5)",NaN,The Bronx,"15,326","13,683","12,797","12,623","11,873","11,647",-226,-1.90%,148.0
3,"161 St-Yankee Stadium (B,D,4)",NaN,The Bronx,"36,176","36,969","35,171","33,398","32,173","31,627",-546,-1.70%,29.0
4,167 St (4),NaN,The Bronx,"10,417","11,942","11,579","9,981","9,822","8,986",-836,-8.50%,178.0
5,"167 St (B,D)",2.0,The Bronx,"13,220","12,890","12,775","12,682","7,529","10,299","2,770",36.80%,163.0


In [9]:
# Do I have the same stops on both the weekend subway dataset and the week day dataset?
subway_df_weekend['Station (alphabetical by borough)'].count()

430

In [21]:
subway_df_weekend.loc['2014': '2018-2019'].astype('float64')

,Station (alphabetical by borough),Unnamed: 1,Boro,2014,2015,2016,2017,2018,2019,2018-2019 Change,Unnamed: 10,2019 Rank


In [22]:
subway_df_weekend.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 430 entries, 1 to 434
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Station (alphabetical by borough)  430 non-null    object 
 1   Unnamed: 1                         76 non-null     float64
 2   Boro                               424 non-null    object 
 3   2014                               426 non-null    object 
 4   2015                               427 non-null    object 
 5   2016                               427 non-null    object 
 6   2017                               430 non-null    object 
 7   2018                               430 non-null    object 
 8   2019                               429 non-null    object 
 9   2018-2019 Change                   430 non-null    object 
 10  Unnamed: 10                        429 non-null    object 
 11  2019 Rank                          424 non-null    float64

In [26]:
subway_df['2014'].str.replace(',', '').astype('float64')

0            NaN
1         3017.0
2        15326.0
3        36176.0
4        10417.0
         ...    
432     873260.0
433        176.0
434    5985905.0
435          NaN
436          NaN
Name: 2014, Length: 437, dtype: float64

In [15]:
subway_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437 entries, 0 to 436
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Station (alphabetical by borough)  435 non-null    object 
 1   Unnamed: 1                         76 non-null     float64
 2   Boro                               424 non-null    object 
 3   2014                               426 non-null    object 
 4   2015                               427 non-null    object 
 5   2016                               427 non-null    object 
 6   2017                               430 non-null    object 
 7   2018                               430 non-null    object 
 8   2019                               429 non-null    object 
 9   2018-2019 Change                   430 non-null    object 
 10  Unnamed: 10                        429 non-null    object 
 11  2019 Rank                          424 non-null    float64

In [13]:
# subway_df_weekend['2014']
subway_df['2014']

0            NaN
1          3,017
2         15,326
3         36,176
4         10,417
         ...    
432      873,260
433          176
434    5,985,905
435          NaN
436          NaN
Name: 2014, Length: 437, dtype: object